In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [3]:
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [5]:
def standard_column_names(df_pd):
      """
      Funtion to standardize column names Upper case the names, Replace spaces with underscore
      (,) with blanks,% with PER and - with undescore
      df: Input dataframe
      df_pd: Output with Standardzied column names
      """
      #df_pd=df.toPandas()
      df_pd.columns = df_pd.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%', 'PER').str.replace('-', '_')
      return df_pd

## Read Files

In [6]:
df_sales_forecasts=pd.read_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Output\sales_volume_forecasts.csv',low_memory=False)

In [7]:
df_sales_forecasts.shape

(135853, 74)

In [8]:
df_sales_forecasts.head()

,PRODUCT_ID,CHANNEL,COUNTRY,PRODUCT_SEASON,PRODUCT_GROUP,PRODUCT_SUBGROUP,PRODUCT_CLASS,PRODUCT_SUBCLASS,PRODUCT_GROUP_CHANNEL_COUNTRY,KEY,ROLLING_WEEK,WEEK_KEY,WEEK,YEAR,CURRENT_PRICE,ORIGINAL_PRICE,PREVIOUS_PRICE,SALES_VOLUME,SALES_VOLUME_LAG_1,SALES_VOLUME_LAG_2,SALES_VOLUME_LAG_3,SALES_VOLUME_LAG_4,SALES_VOLUME_LAG_5,SALES_VOLUME_LAG_6,SALES_VOLUME_LAG_7,SALES_VOLUME_LAG_8,SALES_VOLUME_LAG_9,SALES_VOLUME_LAG_10,SALES_VOLUME_LAG_11,SALES_VOLUME_PRODUCT_SUBGROUP,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_1,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_2,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_3,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_4,SALES_VOLUME_PRODUCT_GROUP,SALES_VOLUME_PRODUCT_GROUP_LAG_1,SALES_VOLUME_PRODUCT_GROUP_LAG_2,SALES_VOLUME_PRODUCT_GROUP_LAG_3,SALES_VOLUME_PRODUCT_GROUP_LAG_4,SALES_VOLUME_PRODUCT_CLASS,SALES_VOLUME_PRODUCT_CLASS_LAG_1,SALES_VOLUME_PRODUCT_CLASS_LAG_2,SALES_VOLUME_PRODUCT_CLASS_LAG_3,SALES_VOLUME_PRODUCT_CLASS_LAG_4,SALES_VOLUME_PRODUCT_SUBCLASS,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_1,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_2,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_3,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_4,EXPANDING_MEAN_SALES_VOLUME_4WEEK,EXPANDING_MEAN_SALES_VOLUME_12WEEK,EXPANDING_MEAN_SALES_VOLUME_8WEEK,EXPANDING_MIN_SALES_VOLUME,EXPANDING_MAX_SALES_VOLUME,EXPANDING_MEAN_SALES_VOLUME,START_DATE,END_DATE,WEEK_OF_MONTH,IS_MONTH_START,IS_MONTH_END,MONTH,IS_YEAR_START,IS_YEAR_END,IS_QUARTER_START,IS_QUARTER_END,WEEK_SIN365_1,WEEK_COS365_1,WEEK_SIN365_2,WEEK_COS365_2,A_F,YEAR_QUARTER,YEAR_MONTH,NO_DAYS_WEEK,DAILY_VOLUME
0,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,26387251-B-STORES,135fc45eStoresB,1,201502,2,2015,18.98,18.98,18.98,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2015-01-05,2015-01-11,2,0,0,1,0,0,0,0,0.239316,0.970942,0.464723,0.885456,Actuals,2015Q1,2015-01,7,0.428571
1,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,26387251-B-STORES,135fc45eStoresB,2,201503,3,2015,18.98,18.98,18.98,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.00,2015-01-12,2015-01-18,3,0,0,1,0,0,0,0,0.354605,0.935016,0.663123,0.748511,Actuals,2015Q1,2015-01,7,0.428571
2,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,26387251-B-STORES,135fc45eStoresB,3,201504,4,2015,18.98,18.98,18.98,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.00,2015-01-19,2015-01-25,4,0,1,1,0,0,0,0,0.464723,0.885456,0.822984,0.568065,Actuals,2015Q1,2015-01,7,0.428571
3,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,26387251-B-STORES,135fc45eStoresB,4,201505,5,2015,18.98,18.98,18.98,4.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,0.0,0.0,3.0,3.0,3.0,0.0,0.0,3.0,3.0,3.0,0.0,0.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.00,2015-01-26,2015-02-01,5,0,1,1,0,0,0,0,0.568065,0.822984,0.935016,0.354605,Actuals,2015Q1,2015-01,7,0.571429
4,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,26387251-B-STORES,135fc45eStoresB,5,201506,6,2015,18.98,18.98,18.98,4.0,4.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,3.0,3.0,0.0,4.0,3.0,3.0,3.0,0.0,4.0,3.0,3.0,3.0,0.0,4.0,3.0,3.0,3.0,0.0,3.4,0.0,0.0,3.0,4.0,3.25,2015-02-02,2015-02-08,1,1,0,2,1,0,0,0,0.663123,0.748511,0.992709,0.120537,Actuals,2015Q1,2015-02,7,0.571429


In [9]:
df_sales_forecasts.isna().sum()

PRODUCT_ID                             0
CHANNEL                                0
COUNTRY                                0
PRODUCT_SEASON                         0
PRODUCT_GROUP                          0
PRODUCT_SUBGROUP                       0
PRODUCT_CLASS                          0
PRODUCT_SUBCLASS                       0
PRODUCT_GROUP_CHANNEL_COUNTRY          0
KEY                                    0
ROLLING_WEEK                           0
WEEK_KEY                               0
WEEK                                   0
YEAR                                   0
CURRENT_PRICE                          0
ORIGINAL_PRICE                         0
PREVIOUS_PRICE                         0
SALES_VOLUME                           0
SALES_VOLUME_LAG_1                     0
SALES_VOLUME_LAG_2                     0
SALES_VOLUME_LAG_3                     0
SALES_VOLUME_LAG_4                     0
SALES_VOLUME_LAG_5                     0
SALES_VOLUME_LAG_6                     0
SALES_VOLUME_LAG

In [10]:
df_coeffecients=pd.read_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Model Output\Sales Volume - Price Coeffecients.csv')

In [11]:
df_coeffecients.shape

(1861, 28)

In [12]:
df_coeffecients.head()

,PRODUCT_ID,CHANNEL,COUNTRY,KEY,INTERCEPT,CURRENT_PRICE_COEFF,WEEK_COEFF,MONTH_COEFF,WEEK_OF_MONTH_COEFF,YEAR_2015_COEFF,YEAR_2016_COEFF,ORIGINAL_PRICE_COEFF,PREVIOUS_PRICE_COEFF,WEEK_SIN365_1_COEFF,WEEK_COS365_1_COEFF,MONTH_SIN365_1_COEFF,MONTH_COS365_1_COEFF,IS_MONTH_START_COEFF,IS_MONTH_END_COEFF,IS_YEAR_START_COEFF,IS_YEAR_END_COEFF,IS_QUARTER_START_COEFF,IS_QUARTER_END_COEFF,MAX_COST_COEFF,COUNTRY_A_COEFF,COUNTRY_B_COEFF,CHANNEL_STORES_COEFF,CHANNEL_ONLINE_COEFF
0,135fc45e,Stores,B,135fc45eStoresB,0.2730,-16.1025,0.0973,-0.8749,-2.8624,18.4377,-18.1647,5.1834,-1.3407,6.3381,7.4422,4.0852,-3.7384,3.3743,0.5932,-4.5646,0.4834,0.0,1.8440,14.7540,0.0,0.2730,0.2730,0.0
1,22125db2,Stores,B,22125db2StoresB,0.0307,-0.3951,-0.0924,-0.0216,0.2946,2.1050,-2.0742,0.5552,-0.0061,1.9781,-0.2645,0.0751,-0.6380,0.4215,0.0946,-1.9122,0.0153,0.0,-0.1678,-0.2963,0.0,0.0307,0.0307,0.0
2,25e07883,Stores,B,25e07883StoresB,-0.1299,3.7930,0.1237,-0.2188,-1.3629,2.7199,-2.8498,-2.1143,-2.6157,-1.6375,1.7350,0.1816,0.5301,-2.0094,0.0368,-0.2443,1.5602,0.0,-0.0161,2.8447,0.0,-0.1299,-0.1299,0.0
3,2729a59e,Stores,B,2729a59eStoresB,224.7794,-77.2324,-5.5986,14.2350,-1.1436,221.8026,2.9768,-27.9810,88.9605,59.3364,-4.8836,-38.3198,-26.7593,-4.2253,-12.4225,-0.0820,74.8760,0.0,6.1957,-57.9243,0.0,224.7794,224.7794,0.0
4,2d16059c,Stores,B,2d16059cStoresB,2.0001,-19.5549,-3.8894,5.8229,1.6706,89.5626,-87.5624,27.1739,-1.0558,25.8114,37.2876,0.4953,1.9409,16.1421,2.3906,17.1923,-12.1875,0.0,4.4118,-14.6662,0.0,2.0001,2.0001,0.0


## Optimize

## Introduce Cost and price deviations

In [13]:
## Almost 67% reductions in price overall as well 

In [14]:
## Assuming Minimum Selling Price is atleast 33% greater than Maximum Cost

In [15]:
df_original=df_sales_forecasts.groupby(['PRODUCT_GROUP_CHANNEL_COUNTRY'])['ORIGINAL_PRICE'].agg({'mean','max','min'}).reset_index()
df_original.columns=['PRODUCT_GROUP_CHANNEL_COUNTRY','MAX_OSP','MEAN_OSP','MIN_OSP']
df_original.head()

,PRODUCT_GROUP_CHANNEL_COUNTRY,MAX_OSP,MEAN_OSP,MIN_OSP
0,26387251-A-STORES,17.233736,11.445,32.72
1,26387251-B-ONLINE,17.109564,11.780,39.25
2,26387251-B-STORES,16.178068,11.110,30.23
3,3bc0de94-B-ONLINE,26.350000,23.480,27.98
4,3bc0de94-B-STORES,26.405000,25.730,27.08


In [16]:
df_previous=df_sales_forecasts.groupby('PRODUCT_GROUP_CHANNEL_COUNTRY')['ORIGINAL_PRICE'].agg({'mean','max','min'}).reset_index()
df_previous.columns=['PRODUCT_GROUP_CHANNEL_COUNTRY','MAX_PSP','MEAN_PSP','MIN_PSP']
df_previous.head()

,PRODUCT_GROUP_CHANNEL_COUNTRY,MAX_PSP,MEAN_PSP,MIN_PSP
0,26387251-A-STORES,17.233736,11.445,32.72
1,26387251-B-ONLINE,17.109564,11.780,39.25
2,26387251-B-STORES,16.178068,11.110,30.23
3,3bc0de94-B-ONLINE,26.350000,23.480,27.98
4,3bc0de94-B-STORES,26.405000,25.730,27.08


In [17]:
df_current=df_sales_forecasts.groupby('PRODUCT_GROUP_CHANNEL_COUNTRY')['CURRENT_PRICE'].agg({'mean','max','min'}).reset_index()
df_current.columns=['PRODUCT_GROUP_CHANNEL_COUNTRY','MAX_CSP','MEAN_CSP','MIN_CSP']
df_current.head()

,PRODUCT_GROUP_CHANNEL_COUNTRY,MAX_CSP,MEAN_CSP,MIN_CSP
0,26387251-A-STORES,15.163353,10.26,32.72
1,26387251-B-ONLINE,15.463752,10.00,39.25
2,26387251-B-STORES,14.373651,10.00,30.23
3,3bc0de94-B-ONLINE,25.409589,16.73,27.98
4,3bc0de94-B-STORES,26.405000,25.73,27.08


In [18]:
df_cost=pd.merge(df_original,df_current,on='PRODUCT_GROUP_CHANNEL_COUNTRY',how='inner')

In [19]:
df_cost.shape

(19, 7)

In [20]:
df_cost.head(20)

,PRODUCT_GROUP_CHANNEL_COUNTRY,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP
0,26387251-A-STORES,17.233736,11.445,32.72,15.163353,10.26,32.72
1,26387251-B-ONLINE,17.109564,11.780,39.25,15.463752,10.00,39.25
2,26387251-B-STORES,16.178068,11.110,30.23,14.373651,10.00,30.23
3,3bc0de94-B-ONLINE,26.350000,23.480,27.98,25.409589,16.73,27.98
4,3bc0de94-B-STORES,26.405000,25.730,27.08,26.405000,25.73,27.08
5,49e9a58f-B-ONLINE,19.794000,16.730,23.48,18.906153,13.13,23.48
6,49e9a58f-B-STORES,19.890000,19.890,19.89,19.245616,16.95,19.89
7,606565a1-A-STORES,14.125493,11.110,20.33,13.247039,10.26,20.33
8,606565a1-B-ONLINE,13.397663,10.880,18.08,12.816924,10.20,18.08
9,606565a1-B-STORES,13.373236,10.880,18.08,12.783121,10.20,18.08


In [21]:
df_cost['percent_change']=(df_cost['MAX_OSP']-df_cost['MIN_CSP'])/(df_cost['MAX_OSP'])

In [22]:
df_cost.head()

,PRODUCT_GROUP_CHANNEL_COUNTRY,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP,percent_change
0,26387251-A-STORES,17.233736,11.445,32.72,15.163353,10.26,32.72,-0.898602
1,26387251-B-ONLINE,17.109564,11.780,39.25,15.463752,10.00,39.25,-1.294039
2,26387251-B-STORES,16.178068,11.110,30.23,14.373651,10.00,30.23,-0.868579
3,3bc0de94-B-ONLINE,26.350000,23.480,27.98,25.409589,16.73,27.98,-0.061860
4,3bc0de94-B-STORES,26.405000,25.730,27.08,26.405000,25.73,27.08,-0.025563


In [23]:
df_cost.describe()

,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP,percent_change
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,17.094489,13.747105,25.261579,15.869307,11.848947,25.261579,-0.511738
std,3.693304,4.448870,6.208691,3.929973,3.990192,6.208691,0.397881
min,13.373236,10.880000,18.080000,12.783121,10.000000,18.080000,-1.405283
25%,15.297380,11.330000,19.890000,13.811240,10.100000,19.890000,-0.741231
50%,16.128409,11.670000,23.480000,14.373651,10.260000,23.480000,-0.439242
75%,17.171650,13.020000,27.980000,15.313553,10.375000,27.980000,-0.252606
max,26.405000,25.730000,39.250000,26.405000,25.730000,39.250000,0.000000


In [24]:
## Maximum cost is in range of 25%, 33%, 40% of MIN CSP
## Minimum cost is in range of 50% of Min CSP

In [25]:
# 25%, 33%, 40%, 

In [26]:
df_original=df_sales_forecasts.groupby(['KEY'])['ORIGINAL_PRICE'].agg({'mean','max','min'}).reset_index()
df_original.columns=['KEY','MAX_OSP','MEAN_OSP','MIN_OSP']
df_original.head()
df_current=df_sales_forecasts.groupby('KEY')['CURRENT_PRICE'].agg({'mean','max','min'}).reset_index()
df_current.columns=['KEY','MAX_CSP','MEAN_CSP','MIN_CSP']
df_current.head()
df_cost=pd.merge(df_original,df_current,on='KEY',how='inner')
df_cost['percent_change']=(df_cost['MAX_OSP']-df_cost['MIN_CSP'])/(df_cost['MAX_OSP'])

In [27]:
df_cost['min_percent']=np.where(df_cost['percent_change']>0.6,0.4,(np.where(df_cost['percent_change']>0.4,0.33,0.25)))

In [28]:
df_cost['MAX_COST']=df_cost['MIN_CSP']*(1-df_cost['min_percent'])

In [29]:
df_cost['MIN_COST']=df_cost['MIN_CSP']*(1-0.5)

In [30]:
df_cost.head()

,KEY,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP,percent_change,min_percent,MAX_COST,MIN_COST
0,10516724OnlineB,14.359616,14.030,14.484,12.042603,11.110,14.030,2.295440e-02,0.25,10.5225,7.015
1,10516724StoresA,15.836000,15.836,15.836,12.483658,11.445,15.836,0.000000e+00,0.25,11.8770,7.918
2,10516724StoresB,14.484000,14.484,14.484,11.908055,11.110,14.484,1.226427e-16,0.25,10.8630,7.242
3,10d8f1f9OnlineB,17.190000,17.190,17.190,17.190000,17.190,17.190,0.000000e+00,0.25,12.8925,8.595
4,10f163a1StoresA,12.234000,12.234,12.234,12.234000,12.234,12.234,0.000000e+00,0.25,9.1755,6.117


In [57]:
df_cost[df_cost['MAX_COST']>=df_cost['MAX_OSP']]

,KEY,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP,percent_change,min_percent,MAX_COST,MIN_COST


In [31]:
df_cost[df_cost['MAX_COST']<=df_cost['MIN_COST']]

,KEY,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP,percent_change,min_percent,MAX_COST,MIN_COST


In [32]:
df_cost[df_cost['MIN_CSP']<=df_cost['MAX_COST']]

,KEY,MAX_OSP,MEAN_OSP,MIN_OSP,MAX_CSP,MEAN_CSP,MIN_CSP,percent_change,min_percent,MAX_COST,MIN_COST


In [33]:
df_cost.isna().sum()

KEY               0
MAX_OSP           0
MEAN_OSP          0
MIN_OSP           0
MAX_CSP           0
MEAN_CSP          0
MIN_CSP           0
percent_change    0
min_percent       0
MAX_COST          0
MIN_COST          0
dtype: int64

### Add Cost as one of the predictors

In [34]:
df_sales=pd.merge(df_sales_forecasts,df_cost[['KEY','MAX_COST']],on='KEY',how='left')

In [35]:
df_sales.shape

(135853, 75)

In [36]:
df_sales['month_sin365_1'] = np.sin(4*np.pi*df_sales['MONTH']/12)
df_sales['month_cos365_1'] = np.cos(4*np.pi*df_sales['MONTH']/12)

## Optimization Data

In [37]:
df_volume=df_sales[['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY', 'KEY', 'ROLLING_WEEK', 'WEEK_KEY',
        'YEAR', 'CURRENT_PRICE', 'ORIGINAL_PRICE', 'PREVIOUS_PRICE',        
        'WEEK_OF_MONTH', 'IS_MONTH_START',
       'IS_MONTH_END', 'MONTH', 'IS_YEAR_START', 'IS_YEAR_END',
       'IS_QUARTER_START', 'IS_QUARTER_END', 'WEEK_SIN365_1', 'WEEK_COS365_1',
       'WEEK_SIN365_2', 'WEEK_COS365_2',
        'SALES_VOLUME', 'A_F', 'MAX_COST','month_sin365_1','month_cos365_1'
       ]]

In [38]:
df_opt=pd.merge(df_volume,df_coeffecients,on=['PRODUCT_ID','CHANNEL','COUNTRY','KEY'],how='left',validate='many_to_one')

In [39]:
df_opt['YEAR_2015']=np.where(df_opt['YEAR']==2015,1,0)
df_opt['YEAR_2016']=np.where(df_opt['YEAR']==2016,1,0)

In [40]:
df_opt['channel_stores']=np.where(df_opt['CHANNEL']=='Stores',1,0)
df_opt['channel_online']=np.where(df_opt['CHANNEL']=='Online',1,0)

In [41]:
df_opt['country_A']=np.where(df_opt['COUNTRY']=='A',1,0)
df_opt['country_B']=np.where(df_opt['COUNTRY']=='B',1,0)

In [42]:
df_opt=standard_column_names(df_opt)

C:\Users\KISHOR~1.KUK\AppData\Local\Temp/ipykernel_23120/1696672.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_pd.columns = df_pd.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%', 'PER').str.replace('-', '_')


In [43]:
df_opt.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS', 'WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY', 'KEY', 'ROLLING_WEEK', 'WEEK_KEY',
       'YEAR', 'CURRENT_PRICE', 'ORIGINAL_PRICE', 'PREVIOUS_PRICE',
       'WEEK_OF_MONTH', 'IS_MONTH_START', 'IS_MONTH_END', 'MONTH',
       'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END',
       'WEEK_SIN365_1', 'WEEK_COS365_1', 'WEEK_SIN365_2', 'WEEK_COS365_2',
       'SALES_VOLUME', 'A_F', 'MAX_COST', 'MONTH_SIN365_1', 'MONTH_COS365_1',
       'INTERCEPT', 'CURRENT_PRICE_COEFF', 'WEEK_COEFF', 'MONTH_COEFF',
       'WEEK_OF_MONTH_COEFF', 'YEAR_2015_COEFF', 'YEAR_2016_COEFF',
       'ORIGINAL_PRICE_COEFF', 'PREVIOUS_PRICE_COEFF', 'WEEK_SIN365_1_COEFF',
       'WEEK_COS365_1_COEFF', 'MONTH_SIN365_1_COEFF', 'MONTH_COS365_1_COEFF',
       'IS_MONTH_START_COEFF', 'IS_MONTH_END_COEFF', 'IS_YEAR_START_COEFF',
       'IS_YEAR_END_COEFF', 'IS_QUARTE

In [44]:
data=df_opt[['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY', 'KEY', 'ROLLING_WEEK', 'WEEK_KEY',
             
       'CURRENT_PRICE', 'ORIGINAL_PRICE', 'PREVIOUS_PRICE',
       'WEEK_OF_MONTH', 'IS_MONTH_START', 'IS_MONTH_END', 'MONTH',
       'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END',
       'WEEK_SIN365_1', 'WEEK_COS365_1', 'MONTH_SIN365_1', 'MONTH_COS365_1',
        'MAX_COST', 'YEAR_2015',
       'YEAR_2016', 'CHANNEL_STORES', 'CHANNEL_ONLINE', 'COUNTRY_A',
       'COUNTRY_B',
             
       'INTERCEPT', 'CURRENT_PRICE_COEFF',
       'WEEK_COEFF', 'MONTH_COEFF', 'WEEK_OF_MONTH_COEFF', 'YEAR_2015_COEFF',
       'YEAR_2016_COEFF', 'ORIGINAL_PRICE_COEFF', 'PREVIOUS_PRICE_COEFF',
       'WEEK_SIN365_1_COEFF', 'WEEK_COS365_1_COEFF', 'MONTH_SIN365_1_COEFF',
       'MONTH_COS365_1_COEFF', 'IS_MONTH_START_COEFF', 'IS_MONTH_END_COEFF',
       'IS_YEAR_START_COEFF', 'IS_YEAR_END_COEFF', 'IS_QUARTER_START_COEFF','IS_QUARTER_END_COEFF', 
       'MAX_COST_COEFF', 'COUNTRY_A_COEFF', 'COUNTRY_B_COEFF',
       'CHANNEL_STORES_COEFF', 'CHANNEL_ONLINE_COEFF',
        
        'SALES_VOLUME', 'A_F']]

In [45]:
data.shape

(135853, 61)

In [46]:
data.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS', 'WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY', 'KEY', 'ROLLING_WEEK', 'WEEK_KEY',
       'CURRENT_PRICE', 'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 'WEEK_OF_MONTH',
       'IS_MONTH_START', 'IS_MONTH_END', 'MONTH', 'IS_YEAR_START',
       'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END', 'WEEK_SIN365_1',
       'WEEK_COS365_1', 'MONTH_SIN365_1', 'MONTH_COS365_1', 'MAX_COST',
       'YEAR_2015', 'YEAR_2016', 'CHANNEL_STORES', 'CHANNEL_ONLINE',
       'COUNTRY_A', 'COUNTRY_B', 'INTERCEPT', 'CURRENT_PRICE_COEFF',
       'WEEK_COEFF', 'MONTH_COEFF', 'WEEK_OF_MONTH_COEFF', 'YEAR_2015_COEFF',
       'YEAR_2016_COEFF', 'ORIGINAL_PRICE_COEFF', 'PREVIOUS_PRICE_COEFF',
       'WEEK_SIN365_1_COEFF', 'WEEK_COS365_1_COEFF', 'MONTH_SIN365_1_COEFF',
       'MONTH_COS365_1_COEFF', 'IS_MONTH_START_COEFF', 'IS_MONTH_END_COEFF',
       'IS_YEAR_START_COEFF', 'IS_YEAR

## Run Optimization

In [47]:
# Run from weeks 61-73

In [48]:
data.head()

,PRODUCT_ID,CHANNEL,COUNTRY,PRODUCT_SEASON,PRODUCT_GROUP,PRODUCT_SUBGROUP,PRODUCT_CLASS,PRODUCT_SUBCLASS,WEEK,PRODUCT_GROUP_CHANNEL_COUNTRY,KEY,ROLLING_WEEK,WEEK_KEY,CURRENT_PRICE,ORIGINAL_PRICE,PREVIOUS_PRICE,WEEK_OF_MONTH,IS_MONTH_START,IS_MONTH_END,MONTH,IS_YEAR_START,IS_YEAR_END,IS_QUARTER_START,IS_QUARTER_END,WEEK_SIN365_1,WEEK_COS365_1,MONTH_SIN365_1,MONTH_COS365_1,MAX_COST,YEAR_2015,YEAR_2016,CHANNEL_STORES,CHANNEL_ONLINE,COUNTRY_A,COUNTRY_B,INTERCEPT,CURRENT_PRICE_COEFF,WEEK_COEFF,MONTH_COEFF,WEEK_OF_MONTH_COEFF,YEAR_2015_COEFF,YEAR_2016_COEFF,ORIGINAL_PRICE_COEFF,PREVIOUS_PRICE_COEFF,WEEK_SIN365_1_COEFF,WEEK_COS365_1_COEFF,MONTH_SIN365_1_COEFF,MONTH_COS365_1_COEFF,IS_MONTH_START_COEFF,IS_MONTH_END_COEFF,IS_YEAR_START_COEFF,IS_YEAR_END_COEFF,IS_QUARTER_START_COEFF,IS_QUARTER_END_COEFF,MAX_COST_COEFF,COUNTRY_A_COEFF,COUNTRY_B_COEFF,CHANNEL_STORES_COEFF,CHANNEL_ONLINE_COEFF,SALES_VOLUME,A_F
0,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,2,26387251-B-STORES,135fc45eStoresB,1,201502,18.98,18.98,18.98,2,0,0,1,0,0,0,0,0.239316,0.970942,0.866025,0.5,14.235,1,0,1,0,0,1,0.273,-16.1025,0.0973,-0.8749,-2.8624,18.4377,-18.1647,5.1834,-1.3407,6.3381,7.4422,4.0852,-3.7384,3.3743,0.5932,-4.5646,0.4834,0.0,1.844,14.754,0.0,0.273,0.273,0.0,3.0,Actuals
1,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,3,26387251-B-STORES,135fc45eStoresB,2,201503,18.98,18.98,18.98,3,0,0,1,0,0,0,0,0.354605,0.935016,0.866025,0.5,14.235,1,0,1,0,0,1,0.273,-16.1025,0.0973,-0.8749,-2.8624,18.4377,-18.1647,5.1834,-1.3407,6.3381,7.4422,4.0852,-3.7384,3.3743,0.5932,-4.5646,0.4834,0.0,1.844,14.754,0.0,0.273,0.273,0.0,3.0,Actuals
2,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,4,26387251-B-STORES,135fc45eStoresB,3,201504,18.98,18.98,18.98,4,0,1,1,0,0,0,0,0.464723,0.885456,0.866025,0.5,14.235,1,0,1,0,0,1,0.273,-16.1025,0.0973,-0.8749,-2.8624,18.4377,-18.1647,5.1834,-1.3407,6.3381,7.4422,4.0852,-3.7384,3.3743,0.5932,-4.5646,0.4834,0.0,1.844,14.754,0.0,0.273,0.273,0.0,3.0,Actuals
3,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,5,26387251-B-STORES,135fc45eStoresB,4,201505,18.98,18.98,18.98,5,0,1,1,0,0,0,0,0.568065,0.822984,0.866025,0.5,14.235,1,0,1,0,0,1,0.273,-16.1025,0.0973,-0.8749,-2.8624,18.4377,-18.1647,5.1834,-1.3407,6.3381,7.4422,4.0852,-3.7384,3.3743,0.5932,-4.5646,0.4834,0.0,1.844,14.754,0.0,0.273,0.273,0.0,4.0,Actuals
4,135fc45e,Stores,B,L,26387251,c3567a18,8face522,641d430c,6,26387251-B-STORES,135fc45eStoresB,5,201506,18.98,18.98,18.98,1,1,0,2,1,0,0,0,0.663123,0.748511,0.866025,-0.5,14.235,1,0,1,0,0,1,0.273,-16.1025,0.0973,-0.8749,-2.8624,18.4377,-18.1647,5.1834,-1.3407,6.3381,7.4422,4.0852,-3.7384,3.3743,0.5932,-4.5646,0.4834,0.0,1.844,14.754,0.0,0.273,0.273,0.0,4.0,Actuals


In [49]:
df_1 = data[data.ROLLING_WEEK >60]
df_1.shape

(24193, 61)

In [50]:
df_1['DATE_KEY']=df_1['KEY']+'-'+df_1['WEEK_KEY'].astype('str')

C:\Users\KISHOR~1.KUK\AppData\Local\Temp/ipykernel_23120/1665971490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['DATE_KEY']=df_1['KEY']+'-'+df_1['WEEK_KEY'].astype('str')


In [59]:
df_1=df_1[df_1['MAX_COST']<=df_1['ORIGINAL_PRICE']]

In [ ]:
df_1=df_1[df_1['ORIGINAL_PRICE']>0]
df_1[df_1['MAX_COST']>df_1['ORIGINAL_PRICE']].head()

In [60]:
df_1.DATE_KEY.unique()

array(['135fc45eStoresB-201610', '135fc45eStoresB-201611',
       '135fc45eStoresB-201612', ..., 'cca085e6StoresB-201620',
       'cca085e6StoresB-201621', 'cca085e6StoresB-201622'], dtype=object)

In [ ]:
#df_1=df_1[(df_1['KEY']=='135fc45eStoresB')|(df_1['KEY']=='cca085e6StoresB')|(df_1['KEY']=='38bc251fStoresB')]
29bfe91eStoresB 

In [61]:
df_1.shape

(24180, 62)

In [63]:
sales_volume=[]
selling_price=[]
date_key=[]
cost_price=[]

### Sales Volume Between 100-125% of Original Sales Forecasts

### Selling Price Between Cost price to Maximum Selling price

In [64]:
for i in df_1['DATE_KEY'].unique():
    
    #print(count_1)
    #count_1 = count_1+1
    
    df_dummy=df_1[df_1['DATE_KEY']==i]
    
    df_dummy.index=range(0,len(df_dummy))
    
    ## Maximum Cost Price
    CP=df_dummy['MAX_COST'][0]
    
    ## Objective is Maximize Total profit
    
    def objective(x):
        return ((x[0]*CP) - (x[0]*x[1]))
    
    def constraint1(x):
        a=  -x[0] 
        
        + df_dummy['CURRENT_PRICE_COEFF']*x[1] 
        + df_dummy['WEEK_COEFF']*df_dummy['WEEK']
        + df_dummy['MONTH_COEFF']*df_dummy['MONTH']
        + df_dummy['WEEK_OF_MONTH_COEFF']*df_dummy['WEEK_OF_MONTH']
        + df_dummy['YEAR_2015_COEFF']*df_dummy['YEAR_2015']
        
        + df_dummy['YEAR_2016_COEFF']*df_dummy['YEAR_2016']
        + df_dummy['ORIGINAL_PRICE_COEFF']*df_dummy['ORIGINAL_PRICE']
        + df_dummy['PREVIOUS_PRICE_COEFF']*df_dummy['PREVIOUS_PRICE']
        + df_dummy['WEEK_SIN365_1_COEFF']*df_dummy['WEEK_SIN365_1']
        
        + df_dummy['WEEK_COS365_1_COEFF']*df_dummy['WEEK_COS365_1']
        + df_dummy['MONTH_SIN365_1_COEFF']*df_dummy['MONTH_SIN365_1']
        + df_dummy['MONTH_COS365_1_COEFF']*df_dummy['MONTH_COS365_1']
        + df_dummy['IS_MONTH_START_COEFF']*df_dummy['IS_MONTH_START']
        
        + df_dummy['IS_MONTH_END_COEFF']*df_dummy['IS_MONTH_END']
        + df_dummy['IS_YEAR_START_COEFF']*df_dummy['IS_YEAR_START']
        + df_dummy['IS_YEAR_END_COEFF']*df_dummy['IS_YEAR_END']
        + df_dummy['IS_QUARTER_START_COEFF']*df_dummy['IS_QUARTER_START']
        + df_dummy['IS_QUARTER_END_COEFF']*df_dummy['IS_QUARTER_END']
        + df_dummy['MAX_COST_COEFF']*df_dummy['MAX_COST']
        + df_dummy['COUNTRY_A_COEFF']*df_dummy['COUNTRY_A']
        + df_dummy['COUNTRY_B_COEFF']*df_dummy['COUNTRY_B']
        
        + df_dummy['CHANNEL_STORES_COEFF']*df_dummy['CHANNEL_STORES_COEFF']
        + df_dummy['CHANNEL_ONLINE_COEFF']*df_dummy['CHANNEL_ONLINE_COEFF']
        + df_dummy['INTERCEPT']
        
        #print(a)
        
        return a   
    
    
    n = 2
    
    x0 = np.zeros(2)
    
    x0[0] = df_dummy['SALES_VOLUME'][0]  # Forecast Sales Volume
    x0[1] = df_dummy['CURRENT_PRICE'][0] # Current Selling Price
    
    
    #x0[2]  = df_dummy['CURRENT_PRICE'][0] ## Current Selling Price
    
    
    b1 = (df_dummy['SALES_VOLUME'][0],1.25*df_dummy['SALES_VOLUME'][0]) #sales bin 80-120% of Forecaasted Value
    b2 = (df_dummy['MAX_COST'][0],df_dummy['ORIGINAL_PRICE'][0]) #price bin - MAX Cost Price , Max Original Price
    
    
    
    bnds = (b1,b2)
    
    con1 = {'type': 'eq', 'fun': constraint1}
    
    cons = ([con1])
    
    solution = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons)
    
    
    x = solution.x
    
    sales_volume.append(x[0]) ## Sales_volume
    selling_price.append(x[1]) ## Price
    date_key.append(i) ## Key
    cost_price.append(CP)
    
df_price_rec=pd.DataFrame()
df_price_rec['date_key']=date_key
df_price_rec['sales_volume']=sales_volume
df_price_rec['selling_price']=selling_price
df_price_rec['cost_price']=cost_price

C:\Users\kishor.kukreja\anaconda3\envs\virtual_Aes\lib\site-packages\scipy\optimize\_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


In [65]:
df_price_rec.shape

(24180, 4)

In [66]:
df_price_rec.head(13)

,date_key,sales_volume,selling_price,cost_price
0,135fc45eStoresB-201610,0.0,14.235,14.235
1,135fc45eStoresB-201611,0.0,14.235,14.235
2,135fc45eStoresB-201612,0.0,14.235,14.235
3,135fc45eStoresB-201613,0.0,14.235,14.235
4,135fc45eStoresB-201614,0.0,14.235,14.235
5,135fc45eStoresB-201615,0.0,14.235,14.235
6,135fc45eStoresB-201616,0.0,14.235,14.235
7,135fc45eStoresB-201617,0.0,14.235,14.235
8,135fc45eStoresB-201618,0.0,14.235,14.235
9,135fc45eStoresB-201619,0.0,14.235,14.235


In [67]:
df_price_rec.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Model Output\Price Optimization.csv',index=False)